In [172]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
pd.options.mode.chained_assignment = None
from sklearn.model_selection import train_test_split
from category_encoders import TargetEncoder

In [173]:
df = pd.read_csv(r"C:\Users\Romar\Google Drive\Master\DE\DE-group-5\Assignment 2\Data\data.csv")
df.head()

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,...,Composure,Marking,StandingTackle,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Release Clause
0,0,158023,L. Messi,31,https://cdn.sofifa.org/players/4/19/158023.png,Argentina,https://cdn.sofifa.org/flags/52.png,94,94,FC Barcelona,...,96.0,33.0,28.0,26.0,6.0,11.0,15.0,14.0,8.0,€226.5M
1,1,20801,Cristiano Ronaldo,33,https://cdn.sofifa.org/players/4/19/20801.png,Portugal,https://cdn.sofifa.org/flags/38.png,94,94,Juventus,...,95.0,28.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0,€127.1M
2,2,190871,Neymar Jr,26,https://cdn.sofifa.org/players/4/19/190871.png,Brazil,https://cdn.sofifa.org/flags/54.png,92,93,Paris Saint-Germain,...,94.0,27.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0,€228.1M
3,3,193080,De Gea,27,https://cdn.sofifa.org/players/4/19/193080.png,Spain,https://cdn.sofifa.org/flags/45.png,91,93,Manchester United,...,68.0,15.0,21.0,13.0,90.0,85.0,87.0,88.0,94.0,€138.6M
4,4,192985,K. De Bruyne,27,https://cdn.sofifa.org/players/4/19/192985.png,Belgium,https://cdn.sofifa.org/flags/7.png,91,92,Manchester City,...,88.0,68.0,58.0,51.0,15.0,13.0,5.0,10.0,13.0,€196.4M


In [174]:
def PreprocessData(df):
    #Remove all NaN rows
    df_preprocessed = df.dropna()
    
    #one hot encoding nationality
    encoder = TargetEncoder()
    df_preprocessed['Nationality Encoded'] = encoder.fit_transform(df_preprocessed['Nationality'], df_preprocessed['Overall'])
    df_preprocessed.pop('Nationality')


    #one hot encoding club (remove NaN first)
    encoder = TargetEncoder()
    df_preprocessed['Club Encoded'] = encoder.fit_transform(df_preprocessed['Club'], df_preprocessed['Overall'])
    df_preprocessed.pop('Club')

    #onehot encoding preferred foot (remove NaN first)\
    encoder = TargetEncoder()
    df_preprocessed['Preferred Foot Encoded'] = encoder.fit_transform(df_preprocessed['Preferred Foot'], df_preprocessed['Overall'])
    df_preprocessed.pop('Preferred Foot')

    #encode Workrate
    encoder = TargetEncoder()
    df_preprocessed['Work Rate Encoded'] = encoder.fit_transform(df_preprocessed['Work Rate'], df_preprocessed['Overall'])
    df_preprocessed.pop('Work Rate')
    
    #encode BodyType
    encoder = TargetEncoder()
    df_preprocessed['Body Type Encoded'] = encoder.fit_transform(df_preprocessed['Body Type'], df_preprocessed['Overall'])
    df_preprocessed.pop('Body Type')

    #encode position
    encoder = TargetEncoder()
    df_preprocessed['Position Encoded'] = encoder.fit_transform(df_preprocessed['Position'], df_preprocessed['Overall'])
    df_preprocessed.pop('Position')


    #Value
    #Wage
    wage = df_preprocessed.Wage
    newwage = []
    for ITEM in wage:
        temp = ITEM.split("€")[1]
        newwage.append(float(temp.split("K")[0]))
    df_preprocessed['Wage'] = newwage
    newwage = []
    
    #Weight
    weight = df_preprocessed.Weight
    newweight = []
    for ITEM  in weight:
        newweight.append(float(ITEM.split("lbs")[0]))
    df_preprocessed['Weight'] = newweight
    newweight = []

    #Height
    newheight = []
    height = df_preprocessed.Height
    for ITEM in height:
        feet, inch = float(ITEM.split("'")[0]), float(ITEM.split("'")[1])
        newheight.append(feet*30.48 + inch*2.54)
    df_preprocessed['Height'] = newheight
    newheight = []
    #All the positions, weet niet wat we daar mee willlen
    positions = ['LS','ST' , 'RS', 'LW', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM','RCM', 'RM','LWB',
                 'LDM','CDM', 'RDM','RWB', 'LB', 'LCB', 'CB', 'RCB','RB']

    for position in positions:
        data = []
        new_data = []
        data = df_preprocessed[position]
        for ITEM in data:
            new_data.append(float(ITEM.split("+")[0]) + float(ITEM.split("+")[1]))
        df_preprocessed[position] = new_data
        new_data = []
    return df_preprocessed
def SplitData(df):
    df.pop('ID')
    df.pop('Unnamed: 0')
    df.pop('Photo')
    df.pop('Flag')
    df.pop('Club Logo')
    df.pop('Release Clause')
    df.pop('Name')
    df.pop('Loaned From')
    df.pop('Real Face')
    df.pop('Contract Valid Until')

    df_pp = PreprocessData(df)

    y = df_pp.pop('Overall')
    X_train, X_test, y_train, y_test = train_test_split(df_pp, y, test_size=0.33, random_state=42)
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = SplitData(df)

In [175]:
x = X_train.columns
for item in x:
    print(item)
    print(X_train[(item)].dtype)

Age
int64
Potential
int64
Value
object
Wage
float64
Special
int64
International Reputation
float64
Weak Foot
float64
Skill Moves
float64
Jersey Number
float64
Joined
object
Height
float64
Weight
float64
LS
float64
ST
float64
RS
float64
LW
float64
LF
object
CF
float64
RF
float64
RW
float64
LAM
float64
CAM
float64
RAM
float64
LM
float64
LCM
float64
CM
float64
RCM
float64
RM
float64
LWB
float64
LDM
float64
CDM
float64
RDM
float64
RWB
float64
LB
float64
LCB
float64
CB
float64
RCB
float64
RB
float64
Crossing
float64
Finishing
float64
HeadingAccuracy
float64
ShortPassing
float64
Volleys
float64
Dribbling
float64
Curve
float64
FKAccuracy
float64
LongPassing
float64
BallControl
float64
Acceleration
float64
SprintSpeed
float64
Agility
float64
Reactions
float64
Balance
float64
ShotPower
float64
Jumping
float64
Stamina
float64
Strength
float64
LongShots
float64
Aggression
float64
Interceptions
float64
Positioning
float64
Vision
float64
Penalties
float64
Composure
float64
Marking
float64
StandingT

In [178]:
X_test.columns

Index(['Age', 'Potential', 'Value', 'Wage', 'Special',
       'International Reputation', 'Weak Foot', 'Skill Moves', 'Jersey Number',
       'Joined', 'Height', 'Weight', 'LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF',
       'RW', 'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM',
       'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'Crossing',
       'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling',
       'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling',
       'GKKicking', 'GKPositioning', 'GKReflexes', 'Nationality Encoded',
       'Club Encoded', 'Preferred Foot Encoded', 'Work Rate Encoded',
       'Body Type Encoded', 'Position Encod